# LeetCode每日一题

## [使数组元素互不相同所需的最少操作次数](https://leetcode.cn/problems/minimum-number-of-operations-to-make-elements-in-array-distinct/description/?envType=daily-question&envId=2025-04-08) 简单

给你一个整数数组 nums，你需要确保数组中的元素互不相同。为此，你可以执行以下操作任意次：

- 从数组的开头移除 3 个元素。如果数组中元素少于 3 个，则移除所有剩余元素。

注意：空数组也视作数组元素互不相同。返回使数组元素互不相同的最少操作次数。

**示例 1：**

> 输入：nums = [1,2,3,2,3,5,7]
> 
> 输出：2
> 
> 解释：
> - 第一次操作：移除前 3 个元素，数组变为 [4, 2, 3, 5, 7]。
> - 第二次操作：再次移除前 3 个元素，数组变为 [5,7]，此时数组中的元素互不相同。
> 因此，答案是 2。

**示例 2：**

> 输入：nums = [4,5,4,4]
> 
> 输出：2
> 
> 解释：
> - 第一次操作：移除前 3 个元素，数组变为 [4]。
> - 第二次操作：移除所有剩余元素，数组变为空。
> 因此，答案是 2。

**示例 3：**

> 输入：nums = [6,7,8,9]
> 
> 输出：0
> 
> 解释：数组中的元素已经互不相同，因此不需要进行任何操作，答案是 0。

**提示：**

- $1 <= nums.length <= 100$
- $1 <= nums[i] <= 100$

In [ ]:
from typing import List

class Solution:
    def minimumOperations(self, nums: List[int]) -> int:
        ans = 0
        n = len(nums)
        idx = 0
        while idx < n:
            if len(nums[idx:]) == len(set(nums[idx:])):
                break
            else:
                idx += 3
                ans += 1
        return ans

## [416. 分割等和子集](https://leetcode.cn/problems/partition-equal-subset-sum/description/?envType=daily-question&envId=2025-04-07) 中等 :star:

给你一个只包含正整数的非空数组 nums。请你判断是否可以将这个数组分割成两个子集，使得两个子集的元素和相等。

**示例 1：**

> 输入：nums = [1, 5, 11, 5]
> 
> 输出：true
> 
> 解释：数组可以分割成 [1, 5, 5] 和 [11]。

**示例 2：**

> 输入：nums = [1, 2, 3, 5]
> 
> 输出：false
> 
> 解释：数组不能分割成两个元素和相等的子集。

**提示：**

- $1 <= nums.length <= 200$
- $1 <= nums[i] <= 100$

In [ ]:
from typing import List

class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        total = sum(nums)
        if total % 2 == 1:
            return False

        # dp[i] = True / False，表示是否可以选出若干个数，使得它们的和为 i
        target = total // 2
        dp = [False] * (target + 1)
        dp[0] = True # base case

        for num in nums:
            # 从大到小遍历，为了避免重复使用同一个数
            for i in range(target, num - 1, -1):
                dp[i] = dp[i] or dp[i - num]
        return dp[-1]

## [368. 最大整除子集](https://leetcode.cn/problems/largest-divisible-subset/description/?envType=daily-question&envId=2025-04-06) 中等 :star:

给你一个由无重复正整数组成的集合 nums，请找出并返回其中最大的整除子集 answer。子集中每一元素对 (answer[i], answer[j]) 都应当满足：
- answer[i] % answer[j] == 0，或
- answer[j] % answer[i] == 0

如果存在多个有效解子集，返回其中任何一个均可。

**示例 1：**

> 输入：nums = [1, 2, 3]
> 
> 输出：[1, 2]
> 
> 解释：[1, 3] 也会被视为正确答案。

**示例 2：**

> 输入：nums = [1, 2, 4, 8]
> 
> 输出：[1, 2, 4, 8]

**提示：**

- $1 <= nums.length <= 1000$
- $1 <= nums[i] <= 2 \times 10^9$
- nums 中的所有整数互不相同

### **为什么这个 DP 方法可以确保子集中的所有元素两两都满足整除关系**，而不仅仅是“当前元素能整除最大元素”。

---

### 1. **关键是数组已经排序**

我们先做了：
```python
nums.sort()
```

这个操作非常关键，它确保了我们只从小数往大数扩展。这样：

- 如果 `a % b == 0`，并且 `b % c == 0`，那么我们能推出 `a % c == 0` 吗？  
  **不一定！**（比如 12 % 6 == 0, 6 % 4 != 0）

但在我们这个问题里，我们不要求任意两数之间整除，而只要子集中每个数能**整除它后面那个数**，构成一条链即可。

### 2. **我们不是要求任意两个数整除，只要求链式整除**

题目要求：
> 找出一个最大子集，**其中任意两个元素 Si 和 Sj，Si % Sj == 0 或 Sj % Si == 0**

这句话其实 **允许我们构造一条整除链**，比如：
```
1 → 2 → 4 → 8 → 24
```

其中任意两个元素，比如 8 和 2，也满足 8 % 2 == 0。为什么？

因为我们通过一个「逐个构造」的过程来添加元素，每次添加都保证了新元素可以整除整个前缀链的尾部。

---

### ✅ 关键保证的来源是“前缀转移”策略

我们构造 dp 的时候：

```python
if nums[i] % nums[j] == 0:
    dp[i] = max(dp[i], dp[j] + 1)
    prev[i] = j
```

这段代码的含义是：  
> “我可以在以 `nums[j]` 结尾的整除子集的基础上，添加上 `nums[i]`。”

由于你只在 `nums[i] % nums[j] == 0` 的前提下才添加，所以你是 **建立在一个合法的整除子集链基础上**扩展的。

---

### 🌈 举个具体例子：

输入：
```python
nums = [1, 2, 3, 4, 8, 9, 72]
```

排序后：
```
[1, 2, 3, 4, 8, 9, 72]
```

你构造出的最大整除子集可能是：
```
[1, 2, 4, 8, 72]
```

这个集合中：

- 2 % 1 == 0
- 4 % 2 == 0
- 8 % 4 == 0
- 72 % 8 == 0

所有元素构成了一条“整除链”，你每次都保证：
> 新加入的数 `nums[i]` 可以整除链的尾部 `nums[j]`

而因为这个链是从小到大扩展的，每个加入的数都会与前面所有数整除。

---

### ✅ 总结一下核心点：

| 保证来源 | 说明 |
|----------|------|
| ✅ 排序 | 确保从小到大构造整除链 |
| ✅ 转移条件 | 只在 `nums[i] % nums[j] == 0` 时才扩展子集 |
| ✅ dp 构建的是「整除链」 | 每个新数加入前面链尾，前面已经合法，所以全链合法 |
| ✅ 回溯时只用 prev[] 构造链 | 没有跳跃式连接，保证“逐个能整除” |

---

### 想象这更像是搭积木🧱：
每一块积木都要能“卡在”前一块的顶部，只有符合整除条件才能接上。你不是强行把所有积木摞一起，而是一块块合法地搭上去，这就是为什么整条链始终合法的原因。

In [ ]:
# TLE
from typing import List

class Solution:
    def largestDivisibleSubset(self, nums: List[int]) -> List[int]:
        nums.sort()  # 排序很关键：保证大的数在后面
        ans = []
        n = len(nums)

        def check(path):
            for i in range(len(path)):
                for j in range(i + 1, len(path)):
                    if path[j] % path[i] != 0 and path[i] % path[j] != 0:
                        return False
            return True

        def dfs(start, path):
            nonlocal ans
            if check(path):
                if len(path) > len(ans):
                    ans = path[:]
            for i in range(start, n):
                path.append(nums[i])
                dfs(i + 1, path)
                path.pop()

        dfs(0, [])
        return ans

In [ ]:
class Solution:
    def largestDivisibleSubset(self, nums: List[int]) -> List[int]:
        nums.sort()
        n = len(nums)
        dp = [1] * n  # dp[i]: 以 nums[i] 结尾的最大整除子集长度
        prev = [-1] * n  # prev[i]: nums[i] 对应的前一个元素索引
        max_idx = 0

        for i in range(n):
            for j in range(i):
                if nums[i] % nums[j] == 0:
                    if dp[j] + 1 > dp[i]:
                        dp[i] = dp[j] + 1
                        prev[i] = j
            if dp[i] > dp[max_idx]:
                max_idx = i

        # 回溯路径
        res = []
        while max_idx != -1:
            res.append(nums[max_idx])
            max_idx = prev[max_idx]
        return res[::-1]

## [1863. 找出所有子集的异或总和再求和](https://leetcode.cn/problems/sum-of-all-subset-xor-totals/description/?envType=daily-question&envId=2025-04-05) 简单 :star:

一个数组的异或总和定义为数组中所有元素按位 XOR 的结果；如果数组为空，则异或总和为 0。

给你一个数组 nums，请你求出 nums 中每个子集的异或总和，计算并返回这些值相加之和。

注意：在本题中，元素相同的不叶子集应多次计数。

**示例 1：**

> 输入：nums = [1, 3]
> 
> 输出：6
> 
> 解释：[1, 3] 共有 4 个子集：
- 空子集的异或总和是 0。
- [1] 的异或总和为 1。
- [3] 的异或总和为 3。
- [1, 3] 的异或总和为 1 XOR 3 = 2。
0 + 1 + 3 + 2 = 6。

**示例 2：**

> 输入：nums = [5, 1, 6]
> 
> 输出：28
> 
> 解释：[5, 1, 6] 共有 8 个子集：
- 空子集的异或总和是 0。
- [5] 的异或总和为 5。
- [1] 的异或总和为 1。
- [6] 的异或总和为 6。
- [5, 1] 的异或总和为 5 XOR 1 = 4。
- [5, 6] 的异或总和为 5 XOR 6 = 3。
- [1, 6] 的异或总和为 1 XOR 6 = 7。
- [5, 1, 6] 的异或总和为 5 XOR 1 XOR 6 = 2。
0 + 5 + 1 + 6 + 4 + 3 + 7 + 2 = 28。

**示例 3：**

> 输入：nums = [3, 4, 5, 6, 7, 8]
> 
> 输出：480
> 
> 解释：每个子集的全部异或总值之和为 480。

**提示：**

- $1 <= nums.length <= 12$
- $1 <= nums[i] <= 20$

In [ ]:
from typing import List

class Solution:
    def subsetXORSum(self, nums: List[int]) -> int:
        n = len(nums)
        ans = 0

        def dfs(val, idx):
            nonlocal ans
            if idx == n:
                ans += val
                return
            
            # 选择当前数字
            dfs(val ^ nums[idx], idx + 1)
            # 不选择当前数字
            dfs(val, idx + 1)

        dfs(0, 0)
        return ans

In [ ]:
class Solution:
    def subsetXORSum(self, nums: List[int]) -> int:
        n = len(nums)
        ans = 0

        def backtrace(start, curr_xor):
            nonlocal ans
            ans += curr_xor
            for i in range(start, n):
                backtrace(i + 1, curr_xor ^ nums[i])

        backtrace(0, 0)
        return ans

## [1123. 最深叶节点的最近公共祖先](https://leetcode.cn/problems/lowest-common-ancestor-of-deepest-leaves/solutions/2421007/zui-shen-xie-jie-dian-de-zui-jin-gong-go-cjzv/?envType=daily-question&envId=2025-04-04) 中等

给你一个有根节点 root 的二叉树，返回它最深的叶节点的最近公共祖先。

**叶节点** 是二叉树中没有子节点的节点。

**树的根节点的深度** 为 0，如果某一节点的深度为 d，那它的子节点的深度就是 d+1。

**最近公共祖先** 的定义：如果我们假定 A 是一组节点 S 的最近公共祖先，S 中的每个节点都在以 A 为根节点的子树中，且 A 的深度达到此条件下可能的最大值。

**示例 1：**

![](https://s3-lc-upload.s3.amazonaws.com/uploads/2018/07/01/sketch1.png)

> 输入：root = [3,5,1,6,2,0,8,null,null,7,4]
> 
> 输出：[2,7,4]
> 
> 解释：我们返回值为 2 的节点，在图中用黄色标记。在图中用蓝色标记的是树的最深的节点。注意，节点 6、0 和 8 也是叶节点，但它们的深度是 2，而节点 7 和 4 的深度是 3。

**示例 2：**

> 输入：root = [1]
> 
> 输出：[1]
> 
> 解释：根节点是树中最深的节点，它是它本身的最近公共祖先。

**示例 3：**

> 输入：root = [0,1,3,null,2]
> 
> 输出：[2]
> 
> 解释：树中最深的叶节点是 2，最近公共祖先是它自己。

**提示：**

- 树中的节点数在 [1, 1000] 的范围内。
- $0 <= Node.val <= 1000$
- 每个节点的值都是独一无二的。

In [ ]:
from typing import Optional

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def lcaDeepestLeaves(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        self.leaves = []

        def dfs(node, depth):
            if not node:
                return
            if not node.left and not node.right:
                self.leaves.append((node, depth))
            dfs(node.left, depth + 1)
            dfs(node.right, depth + 1)
        
        dfs(root, 0)
        if not self.leaves:
            return None  # This case should not occur as per problem constraints
        
        max_depth = max(leaf[1] for leaf in self.leaves)
        deepest_leaves = [leaf[0] for leaf in self.leaves if leaf[1] == max_depth]
        
        if len(deepest_leaves) == 1:
            return deepest_leaves[0]
        
        def findParent(node, p, q):
            if not node or p == node or q == node:
                return node
            left = findParent(node.left, p, q)
            right = findParent(node.right, p, q)
            if left and right:
                return node
            return left if left else right
        
        current_lca = deepest_leaves[0]
        for node in deepest_leaves[1:]:
            current_lca = findParent(root, current_lca, node)
        return current_lca

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def lcaDeepestLeaves(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        ans = None
        max_depth = -1
        
        def dfs(node, depth):
            nonlocal ans, max_depth
            if not node:
                max_depth = max(max_depth, depth)
                return depth
            left_max_depth = dfs(node.left, depth + 1)
            right_max_depth = dfs(node.right, depth + 1)
            if left_max_depth == right_max_depth == max_depth:
                ans = node
            return max(left_max_depth, right_max_depth)
        
        dfs(root, 0)
        return ans

## [有序三元组中的最大值 II](https://leetcode.cn/problems/maximum-value-of-an-ordered-triplet-ii/?envType=daily-question&envId=2025-04-03) 中等

给你一个下标从 0 开始的整数数组 nums。

请你从所有满足 i < j < k 的下标三元组 (i, j, k) 中，找出并返回下标三元组的最大值。如果所有满足条件的三元组的值都是负数，则返回 0。

下标三元组 (i, j, k) 的值等于 (nums[i] - nums[j]) * nums[k]。

**示例 1：**

> 输入：nums = [12, 6, 1, 2, 7]
> 
> 输出：77
> 
> 解释：下标三元组 (0, 2, 4) 的值是 (nums[0] - nums[2]) * nums[4] = 77。可以证明不存在值大于 77 的有序下标三元组。

**示例 2：**

> 输入：nums = [1, 10, 3, 4, 19]
> 
> 输出：133
> 
> 解释：下标三元组 (1, 2, 4) 的值是 (nums[1] - nums[2]) * nums[4] = 133。可以证明不存在值大于 133 的有序下标三元组。

**示例 3：**

> 输入：nums = [1, 2, 3]
> 
> 输出：0
> 
> 解释：唯一的下标三元组 (0, 1, 2) 的值是一个负数，(nums[0] - nums[1]) * nums[2] = -3。因此，答案是 0。

**提示：**

- $3 <= nums.length <= 10^5$
- $1 <= nums[i] <= 10^6$

In [ ]:
from typing import List

class Solution:
    def maximumTripletValue(self, nums: List[int]) -> int:
        n = len(nums)
        max_left = [0] * n
        max_right = [0] * n
        ans = 0

        # 预处理左侧最大值
        max_left[0] = nums[0]
        for i in range(1, n):
            max_left[i] = max(max_left[i - 1], nums[i])
        
        # 预处理右侧最大值
        max_right[n - 1] = nums[n - 1]
        for i in range(n - 2, -1, -1):
            max_right[i] = max(max_right[i + 1], nums[i])

        for i in range(1, n - 1):
            ans = max(ans, (max_left[i - 1] - nums[i]) * max_right[i + 1])

        return ans

## [有序三元组中的最大值 I](https://leetcode.cn/problems/maximum-value-of-an-ordered-triplet-i/description/?envType=daily-question&envId=2025-04-02) 简单

给你一个下标从 0 开始的整数数组 nums。

请你从所有满足 i < j < k 的下标三元组 (i, j, k) 中，找出并返回下标三元组的最大值。如果所有满足条件的三元组的值都是负数，则返回 0。

下标三元组 (i, j, k) 的值等于 (nums[i] - nums[j]) * nums[k]。

**示例 1：**

> 输入：nums = [12, 6, 1, 2, 7]
> 
> 输出：77
> 
> 解释：下标三元组 (0, 2, 4) 的值是 (nums[0] - nums[2]) * nums[4] = 77。可以证明不存在值大于 77 的有序下标三元组。

**示例 2：**

> 输入：nums = [1, 10, 3, 4, 19]
> 
> 输出：133
> 
> 解释：下标三元组 (1, 2, 4) 的值是 (nums[1] - nums[2]) * nums[4] = 133。可以证明不存在值大于 133 的有序下标三元组。

**示例 3：**

> 输入：nums = [1, 2, 3]
> 
> 输出：0
> 
> 解释：唯一的下标三元组 (0, 1, 2) 的值是一个负数，(nums[0] - nums[1]) * nums[2] = -3。因此，答案是 0。

**提示：**

- $3 <= nums.length <= 100$
- $1 <= nums[i] <= 10^6$

In [ ]:
from typing import List

class Solution:
    def maximumTripletValue(self, nums: List[int]) -> int:
        ans = 0
        n = len(nums)
        for i in range(n - 2):
            for j in range(i + 1, n - 1):
                for k in range(j + 1, n):
                    ans = max(ans, (nums[i] - nums[j]) * nums[k])
        return ans

In [ ]:
class Solution:
    def maximumTripletValue(self, nums: List[int]) -> int:
        ans = 0
        n = len(nums)
        if n < 3:
            return 0

        # 预计算 max_left[i]: 在 i 之前[:i + 1]的最大 nums[i]
        max_left = [0] * n
        max_left[0] = nums[0]
        for i in range(1, n):
            max_left[i] = max(max_left[i - 1], nums[i])

        # 预计算 max_right[i]: 在 i 之后[i:]的最大 nums[i]
        max_right = [0] * n
        max_right[n - 1] = nums[n - 1]
        for i in range(n - 2, -1, -1):
            max_right[i] = max(max_right[i + 1], nums[i])

        for j in range(1, n - 1):
            max_i = max_left[j - 1]
            max_k = max_right[j + 1]
            ans = max(ans, (max_i - nums[j]) * max_k)

        return ans

## [解决智力问题](https://leetcode.cn/problems/solving-questions-with-brainpower/?envType=daily-question&envId=2025-04-01) 中等

给你一个二维整数数组 questions，其中 $questions[i] = [points_i, brainpower_i]$。

你需要按顺序（从问题 0 开始依次解决）针对每个问题选择解决或跳过。解决问题 i 将获得 $points_i$ 分，但接下来的 $brainpower_i$ 个问题无法解决。跳过问题 i 则可以继续处理下一个问题。

返回这场考试中你能获得的最高分数。

**示例 1：**

> 输入：questions = [[3,2], [4,3], [4,4], [2,5]]
> 
> 输出：5
> 
> 解释：解决问题 0 和 3 得到最高分。
- 解决问题 0：获得 3 分，但接下来 2 个问题无法解决。
- 解决问题 3：获得 2 分。
总得分为 3 + 2 = 5。

**示例 2：**

> 输入：questions = [[1,1], [2,2], [3,3], [4,4], [5,5]]
> 
> 输出：7
> 
> 解释：解决问题 1 和 4 得到最高分。
- 跳过问题 0。
- 解决问题 1：获得 2 分，但接下来 2 个问题无法解决。
- 解决问题 4：获得 5 分。
总得分为 2 + 5 = 7。

**提示：**

- $1 <= questions.length <= 10^5$
- $questions[i].length == 2$
- $1 <= points_i, brainpower_i <= 10^5$

In [ ]:
from typing import List

class Solution:
    def mostPoints(self, questions: List[List[int]]) -> int:
        # dp[i] = max(dp[i + questions[i][1] + 1] + questions[i][0], dp[i + 1])
        n = len(questions)
        dp = [0] * (n + 1)
        for i in range(n - 1, -1, -1):
            points, brainpower = questions[i]
            next_index = i + brainpower + 1
            solve = points + (dp[next_index] if next_index <= n else 0)
            skip = dp[i + 1]
            dp[i] = max(solve, skip)
        return dp[0]

## [向字符串添加空格](https://leetcode.cn/problems/adding-spaces-to-a-string/description/?envType=daily-question&envId=2025-03-30) 中等

给你一个下标从 0 开始的字符串 s ，以及一个下标从 0 开始的整数数组 spaces 。

数组 spaces 描述原字符串中需要添加空格的下标。每个空格都应该插入到给定索引处的字符值 之前 。

- 例如，s = "EnjoyYourCoffee" 且 spaces = [5, 9] ，那么我们需要在 'Y' 和 'C' 之前添加空格，这两个字符分别位于下标 5 和下标 9 。因此，最终得到 "Enjoy Your Coffee" 。

请你添加空格，并返回修改后的字符串。

**示例 1：**

> 输入：s = "LeetcodeHelpsMeLearn", spaces = [8, 13, 15]
> 
> 输出："Leetcode Helps Me Learn"
> 
> 解释：下标 8、13 和 15 对应 "LeetcodeHelpsMeLearn" 中加粗斜体字符。接着在这些字符前添加空格。

**示例 2：**

> 输入：s = "icodeinpython", spaces = [1, 5, 7, 9]
> 
> 输出："i code in py thon"
> 
> 解释：下标 1、5、7 和 9 对应 "icodeinpython" 中加粗斜体字符。接着在这些字符前添加空格。

**示例 3：**

> 输入：s = "spacing", spaces = [0, 1, 2, 3, 4, 5]
> 
> 输出：" s p a c i n g"
> 
> 解释：字符串的第一个字符前可以添加空格。

**提示：**

- $1 <= s.length <= 3 \times 10^5$
- s 仅由大小写英文字母组成
- $1 <= spaces.length <= 3 \times 10^5$
- $0 <= spaces[i] <= s.length - 1$
- spaces 中的所有值严格递增

In [ ]:
from typing import List

class Solution:
    def addSpaces(self, s: str, spaces: List[int]) -> str:
        chars = []
        for ch in s:
            chars.append(ch)

        for space in spaces:
            chars[space] = ' ' + chars[space]

        return ''.join(chars)

## [2716. 最小化字符串长度](https://leetcode.cn/problems/minimize-string-length/description/?envType=daily-question&envId=2025-03-28) 简单

给你一个下标从 0 开始的字符串 s ，重复执行下述操作任意次：

- 在字符串中选出一个下标 i ，并使 c 为字符串下标 i 处的字符。并在 i 左侧（如果有）和右侧（如果有）各删除一个距离 i 最近的字符 c 。

请你通过执行上述操作任意次，使 s 的长度最小化。

返回一个表示最小化字符串的长度的整数。

**示例 1：**

> 输入：s = "aaabc"
> 
> 输出：3
> 
> 解释：在这个示例中，s 等于 "aaabc"。我们可以选择位于下标 1 处的字符 'a' 开始。接着删除下标 1 左侧最近的那个 'a'（位于下标 0）以及下标 1 右侧最近的那个 'a'（位于下标 2）。执行操作后，字符串变为 "abc"。继续对字符串执行任何操作都不会改变其长度。因此，最小化字符串的长度是 3。

**示例 2：**

> 输入：s = "cbbd"
> 
> 输出：3
> 
> 解释：我们可以选择位于下标 1 处的字符 'b' 开始。下标 1 左侧不存在字符 'b'，但右侧存在一个字符 'b'（位于下标 2），所以会删除位于下标 2 的字符 'b'。执行操作后，字符串变为 "cbd"。继续对字符串执行任何操作都不会改变其长度。因此，最小化字符串的长度是 3。

**示例 3：**

> 输入：s = "ddaadaa"
> 
> 输出：2
> 
> 解释：我们可以选择位于下标 1 处的字符 'd' 开始。接着删除下标左侧最近的那个 'd'（位于下标 0）以及下标 1 右侧最近的那个 'd'（位于下标 2）。执行操作后，字符串变为 "daaa"。继续对新字符串执行操作，可以选择位于下标 2 的字符 'a'。接着删除下标 2 左侧最近的那个 'a'（位于下标 1）以及下标 2 右侧最近的那个 'a'（位于下标 3）。执行操作后，字符串变为 "da"。继续对字符串执行任何操作都不会改变其长度。因此，最小化字符串的长度是 2。

**提示：**

- $1 <= s.length <= 100$
- $s$ 仅由小写英文字母组成

In [6]:
class Solution:
    def minimizedStringLength(self, s: str) -> int:
        return len(set(s))

In [4]:
class Solution:
    def minimizedStringLength(self, s: str) -> int:
        n = len(s)
        chars = list(s)
        for i in range(n):
            if i == 0:
                # 将右侧最近的字符 s[i] 删除
                for j in range(i + 1, n):
                    if chars[i] != '-' and chars[j] == chars[i]:
                        chars[j] = '-' # 标记为删除
                        break
            elif i == n - 1:
                # 将左侧最近的字符 s[i] 删除
                for j in range(i - 1, -1, -1):
                    if chars[i] != '-' and chars[j] == chars[i]:
                        chars[j] = '-'
                        break
            else:
                # 将左侧最近的字符 s[i] 删除
                for j in range(i - 1, -1, -1):
                    if chars[i] != '-' and chars[j] == chars[i]:
                        chars[j] = '-'
                        break
                # 将右侧最近的字符 s[i] 删除
                for j in range(i + 1, n):
                    if chars[i] != '-' and chars[j] == chars[i]:
                        chars[j] = '-' # 标记为删除
                        break
        count = 0
        for ch in chars:
            if ch != '-':
                count += 1
        return count

In [7]:
print(Solution().minimizedStringLength("aaabc")) # 3

3


## [2712. 使所有字符相等的最小成本](https://leetcode.cn/problems/minimum-cost-to-make-all-characters-equal/description/?envType=daily-question&envId=2025-03-27) 中等 :star:

给你一个下标从 0 开始、长度为 n 的二进制字符串 s，你可以对其执行两种操作：

- 选中一个下标 i 并且反转从下标 0 到下标 i（包括下标 0 和下标 i）的所有字符，成本为 i + 1。
- 选中一个下标 i 并且反转从下标 i 到下标 n - 1（包括下标 i 和下标 n - 1）的所有字符，成本为 n - i。

返回使字符串内所有字符相等需要的最小成本。

**反转**字符意味着：如果原来的值是 '0'，则反转后变为 '1'，反之亦然。

**示例 1：**

> 输入：s = "0011"
> 
> 输出：2
> 
> 解释：执行第二种操作，选中下标 i = 2，可以得到 s = "0000"，成本为 2。可以证明 2 是使所有字符相等的最小成本。

**示例 2：**

> 输入：s = "010101"
> 
> 输出：9
> 
> 解释：执行第一种操作，选中下标 i = 2，可以得到 s = "101101"，成本为 3。
> 
> 执行第一种操作，选中下标 i = 1，可以得到 s = "011101"，成本为 2。
> 
> 执行第一种操作，选中下标 i = 0，可以得到 s = "111101"，成本为 1。
> 
> 执行第二种操作，选中下标 i = 4，可以得到 s = "111110"，成本为 2。
> 
> 执行第二种操作，选中下标 i = 5，可以得到 s = "111111"，成本为 1。
> 
> 使所有字符相等的总成本等于 9。可以证明 9 是使所有字符相等的最小成本。

**提示：**

- $1 <= s.length == n <= 10^5$
- $s[i]$ 为 '0' 或 '1'

可以用一个非常简洁的思路来解决这个问题：当我们从左到右扫字符串时，每遇到相邻字符不相等的位置 i (即 s[i] != s[i+1])，就说明这里存在“边界”需要我们花费一次翻转操作去把这个边界消除（把这两段统一为相同字符）。对于这个边界，我们可以有两种做法：

1. 翻转从下标 0 到 i 的前缀，成本为 (i + 1)  
2. 翻转从下标 i+1 到 n-1 的后缀，成本为 (n - (i+1)) = (n - i - 1)

我们只需要选择较小的那个成本即可。因此，每当 s[i] != s[i+1]，就把答案加上 `min(i+1, n - i - 1)`。

**思路解析：**

- 一次操作可以把「边界」两侧翻转成同一种字符；  
- 任何一次翻转只会影响到当前及以后的边界，不会反复干扰已处理过的边界；  
- 因此只要把所有不相等的邻接处（边界）都“修复”一次，就能让整串变为同一个字符；  
- 对每个边界，只需选“翻转前缀”或者“翻转后缀”中成本更低的一种，累加到最终答案即可。

In [ ]:
class Solution:
    def minimumCost(self, s: str) -> int:
        n = len(s)
        cost = 0
        for i in range(n - 1):
            if s[i] != s[i + 1]:
                cost += min(i + 1, n - i - 1)
        return cost

## [对角线上不同值的数量差](https://leetcode.cn/problems/difference-of-number-of-distinct-values-on-diagonals/description/?envType=daily-question&envId=2025-03-25) 中等

给你一个下标从 0 开始、大小为 m x n 的二维矩阵 grid，请你求解大小同样为 m x n 的答案矩阵 answer。

矩阵 answer 中每个单元格 (r, c) 的值可以按下述方式进行计算：

- 令 topLeft[r][c] 为矩阵 grid 中单元格 (r, c) 左上角对角线上不同值的数量。
- 令 bottomRight[r][c] 为矩阵 grid 中单元格 (r, c) 右下角对角线上不同值的数量。
- 然后 answer[r][c] = |topLeft[r][c] - bottomRight[r][c]|。

返回矩阵 answer。

矩阵对角线是从最顶行或最左列的某个单元格开始，向右下方向走到矩阵末尾的对角线。

如果单元格 (r1, c1) 和单元格 (r, c) 属于同一条对角线且 r1 < r，则单元格 (r1, c1) 属于单元格 (r, c) 的左上对角线。类似地，可以定义右下对角线。

**示例 1：**

![](https://assets.leetcode.com/uploads/2023/04/19/ex2.png)

> 输入：grid = [[1,2,3],[3,1,5],[3,2,1]]
> 
> 输出：[[1,1,0],[1,0,1],[0,1,1]]
> 
> 解释：
> - 单元格 (0,0) 的右下对角线包含 [1,1]，左上对角线包含 []。对应答案是 |1 - 0| = 1。
> - 单元格 (1,2) 的右下对角线包含 []，左上对角线包含 [2]。对应答案是 |0 - 1| = 1。
> - 单元格 (1,1) 的右下对角线包含 [1]，左上对角线包含 [1]。对应答案是 |1 - 1| = 0。
> 其他单元格的对应答案也可以按照这样的流程进行计算。

**示例 2：**

> 输入：grid = [[1]]
> 
> 输出：[[0]]
> 
> 解释：
> - 单元格 (0,0) 的右下对角线包含 []，左上对角线包含 []。对应答案是 |0 - 0| = 0。

**提示：**

- m == grid.length
- n == grid[i].length
- 1 <= m, n, grid[i][j] <= 50

In [23]:
from typing import List
from collections import defaultdict

class Solution:
    def differenceOfDistinctValues(self, grid: List[List[int]]) -> List[List[int]]:
        m, n = len(grid), len(grid[0])
        top_left = [[0] * n for _ in range(m)]
        bottom_right = [[0] * n for _ in range(m)]
        diagonals = defaultdict(list)
        
        # 将单元格按对角线分组
        for i in range(m):
            for j in range(n):
                diagonals[i - j].append((i, j))
        
        # 预处理每个对角线的左上和右下不同值数量
        for key in diagonals:
            elements = diagonals[key]
            
            # 计算左上部分的不同值数目
            seen = set()
            for i, j in elements:
                top_left[i][j] = len(seen)
                seen.add(grid[i][j])
            
            # 计算右下部分的不同值数目
            seen = set()
            for i, j in reversed(elements):
                bottom_right[i][j] = len(seen)
                seen.add(grid[i][j])
        
        # 构建结果矩阵
        ans = [[0] * n for _ in range(m)]
        for i in range(m):
            for j in range(n):
                ans[i][j] = abs(top_left[i][j] - bottom_right[i][j])
        
        return ans

In [24]:
from typing import List

class Solution:
    def differenceOfDistinctValues(self, grid: List[List[int]]) -> List[List[int]]:
        m, n = len(grid), len(grid[0])
        ans = [[0 for _ in range(n)] for _ in range(m)]
        
        def caculate(i: int, j: int) -> int:
            # 左上角
            topLeftNums = set()
            left_i, left_j = i - 1, j - 1
            while left_i >= 0 and left_j >= 0:
                topLeftNums.add(grid[left_i][left_j])
                left_i -= 1
                left_j -= 1

            # 右下角
            bottomRightNums = set()
            right_i, right_j = i + 1, j + 1
            while right_i < m and right_j < n:
                bottomRightNums.add(grid[right_i][right_j])
                right_i += 1
                right_j += 1

            topLeft, bottomRight = 0, 0
            for num in topLeftNums:
                if num not in bottomRightNums:
                    topLeft += 1
            for num in bottomRightNums:
                if num not in topLeftNums:
                    bottomRight += 1

            return abs(topLeft - bottomRight)
        
        for i in range(m):
            for j in range(n):
                ans[i][j] = caculate(i, j)

        return ans

## [k-avoiding 数组的最小总和](https://leetcode.cn/problems/determine-the-minimum-sum-of-a-k-avoiding-array/description/?envType=daily-question&envId=2025-03-26) 中等

给你两个整数 n 和 k。

对于一个由不同正整数组成的数组，如果其中不存在任何求和等于 k 的不同元素对，则称其为 k-avoiding 数组。返回长度为 n 的 k-avoiding 数组的可能的最小总和。

**示例 1：**

> 输入：n = 5, k = 4
> 
> 输出：18
> 
> 解释：设若 k-avoiding 数组为 [1,2,4,5,6]，其元素总和为 18。
> 
> 可以证明不存在总和小于 18 的 k-avoiding 数组。

**示例 2：**

> 输入：n = 2, k = 6
> 
> 输出：3
> 
> 解释：可以构造数组 [1,2]，其元素总和为 3。
> 
> 可以证明不存在总和小于 3 的 k-avoiding 数组。

**提示：**
- $1 <= n, k <= 50$

In [25]:
class Solution:
    def minimumSum(self, n: int, k: int) -> int:
        ans = []
        
        for i in range(1, k // 2 + 1):
            if len(ans) < n:
                ans.append(i)
            if len(ans) == n:
                break
        num = k
        while len(ans) < n:
            ans.append(num)
            num += 1
            
        return sum(ans)

In [26]:
class Solution:
    def minimumSum(self, n: int, k: int) -> int:
        m = (k - 1) // 2
        if n <= m:
            return n * (n + 1) // 2
        rest = n - m
        if k % 2 == 0:
            part1 = m * (m + 1) // 2
            part2 = k // 2
            part3 = (rest - 1) * k + (rest - 1) * (rest - 2) // 2
            return part1 + part2 + part3
        else:
            part1 = m * (m + 1) // 2
            part2 = rest * k + rest * (rest - 1) // 2
            return part1 + part2

## 图书整理 I

书店店员有一张链表形式的书单，每个节点代表一本书，节点中的值表示书的编号。为更方便整理书架，店员需要将书单倒过来排列，就可以从最后一本书开始整理，逐一将书放回到书架上。请倒序返回这个书单链表。

示例 1：
```
输入：head = [3,6,4,1]

输出：[1,4,6,3]
```
提示：
```
0 <= 链表长度 <= 10000
```

In [27]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [28]:
from typing import Optional, List


def reverseBookList(self, head: Optional[ListNode]) -> List[int]:
    res = []
    while head:
        res.append(head.val)
        head = head.next
    return res[::-1]

In [29]:
reverseBookList([3, 6, 4, 1], ListNode(3, ListNode(6, ListNode(4, ListNode(1)))))

[1, 4, 6, 3]

## 删除链表节点
给定单向链表的头指针和一个要删除的节点的值，定义一个函数删除该节点。
返回删除后的链表的头节点。
示例 1:
```
输入: head = [4,5,1,9], val = 5
输出: [4,1,9]
解释: 给定你链表中值为5的第二个节点，那么在调用了你的函数之后，该链表应变为 4 -> 1 -> 9.
```
示例 2:
```
输入: head = [4,5,1,9], val = 1
输出: [4,5,9]
解释: 给定你链表中值为 1 的第三个节点，那么在调用了你的函数之后，该链表应变为 4 -> 5 -> 9.
```
说明：

* 题目保证链表中节点的值互不相同
* 若使用 C 或 C++ 语言，你不需要 free 或 delete 被删除的节点

In [30]:
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None


def deleteNode(head: ListNode, val: int) -> ListNode:
    if head.val == val:
        return head.next
    pre, cur = head, head.next
    if cur:
        while cur.val != val:
            pre = cur
            cur = cur.next
        pre.next = cur.next
    return head

In [31]:
a = ListNode(4)
b = ListNode(5)
c = ListNode(1)
d = ListNode(9)
a.next = b
b.next = c
c.next = d
res = deleteNode(head=a, val=5)
while res:
    print(res.val)
    res = res.next

4
1
9


In [32]:
a = ListNode(4)
b = ListNode(5)
c = ListNode(1)
d = ListNode(9)
a.next = b
b.next = c
c.next = d
res = deleteNode(head=a, val=1)
while res:
    print(res.val)
    res = res.next

4
5
9


## 训练计划 III
给定一个头节点为 head 的单链表用于记录一系列核心肌群训练编号，请将该系列训练编号 倒序 记录于链表并返回。

示例 1：
```
输入：head = [1,2,3,4,5]
输出：[5,4,3,2,1]
```
示例 2：
```
输入：head = [1,2]
输出：[2,1]
```
示例 3：
```
输入：head = []
输出：[]
```
提示：
* 链表中节点的数目范围是 [0, 5000]
* -5000 <= Node.val <= 5000

In [33]:
from typing import Optional, List


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next


def trainingPlanIII(head: Optional[ListNode]) -> Optional[ListNode]:
    if not head:
        return head
    res = []
    while head:
        res.append(head.val)
        head = head.next
    head = ListNode(res[0])
    for i in range(1, len(res)):
        head = ListNode(res[i], head)
    return head

In [34]:
res = trainingPlanIII(ListNode(1, ListNode(2, ListNode(3, ListNode(4, ListNode(5))))))
while res:
    print(res.val)
    res = res.next

5
4
3
2
1


In [35]:
def PlanIII(head: Optional[ListNode]) -> Optional[ListNode]:
    if not head:
        return head
    dummy = ListNode(0)
    dummy.next = head
    pre = dummy
    cur = head
    while cur.next:
        tmp = cur.next
        cur.next = tmp.next
        tmp.next = pre.next
        pre.next = tmp
    return dummy.next

In [36]:
res = PlanIII(ListNode(1, ListNode(2, ListNode(3, ListNode(4, ListNode(5))))))
while res:
    print(res.val)
    res = res.next

5
4
3
2
1


## 训练计划 II
给定一个头节点为 head 的链表用于记录一系列核心肌群训练项目编号，请查找并返回倒数第 cnt 个训练项目编号。

示例 1：
```
输入：head = [2,4,7,8], cnt = 1
输出：8
```
提示：
* 1 <= head.length <= 100
* 0 <= head[i] <= 100
* 1 <= cnt <= head.length

In [37]:
from typing import Optional, List


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next


def trainingPlanII(head: Optional[ListNode], cnt: int) -> Optional[ListNode]:
    if not head:
        return head
    res = []
    while head:
        res.append(head)
        head = head.next
    return res[-cnt]

In [38]:
res = trainingPlanII(ListNode(2, ListNode(4, ListNode(7, ListNode(8)))), 2)
while res:
    print(res.val)
    res = res.next

7
8


In [39]:
from typing import Optional, List


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next


def PlanII(head: Optional[ListNode], cnt: int) -> Optional[ListNode]:
    left, right = head, head
    k = 0
    while k < cnt:
        right = right.next
        k += 1
    while right:
        left = left.next
        right = right.next
    return left

In [40]:
res = PlanII(ListNode(2, ListNode(4, ListNode(7, ListNode(8)))), 2)
while res:
    print(res.val)
    res = res.next

7
8


## 2960. 统计已测试设备

In [41]:
def countTestedDevices(self, batteryPercentages: List[int]) -> int:
    count = 0
    n = len(batteryPercentages)
    for i in range(n):
        if batteryPercentages[i] > 0:
            count += 1
            for j in range(i, n):
                batteryPercentages[j] = max(0, batteryPercentages[j] - 1)
    return count

In [42]:
def countTestedDevices(self, batteryPercentages: List[int]) -> int:
    need = 0
    for battery in batteryPercentages:
        if battery > need:
            need += 1
    return need

## 2391. 收集垃圾的最少总时间

In [43]:
def garbageCollection(self, garbage: List[str], travel: List[int]) -> int:
    n = len(garbage)
    res = 0
    flag = False
    for i in range(n - 1, -1, -1):
        if 'M' in garbage[i]:
            res += garbage[i].count('M') # 收拾垃圾的时间
            # 计算行驶时间
            if flag == False:
                flag = True
                for j in range(i):
                    res += travel[j] # 从上一个垃圾点到当前垃圾点的时间
    print(res)
    flag = False
    for i in range(n - 1, -1, -1):
        if 'P' in garbage[i]:
            res += garbage[i].count('P') # 收拾垃圾的时间
            # 计算行驶时间
            if flag == False:
                flag = True
                for j in range(i):
                    res += travel[j] # 从上一个垃圾点到当前垃圾点的时间
    print(res)
    flag = False
    for i in range(n - 1, -1, -1):
        if 'G' in garbage[i]:
            res += garbage[i].count('G') # 收拾垃圾的时间
            # 计算行驶时间
            if flag == False:
                flag = True
                for j in range(i):
                    res += travel[j] # 从上一个垃圾点到当前垃圾点的时间
    return res

In [44]:
def garbageCollection(self, garbage: List[str], travel: List[int]) -> int:
    n = len(garbage)
    distance = {} # 用于存储不同的垃圾车移动的距离
    res = 0
    cur_dis = 0
    for i in range(n):
        res += len(garbage[i]) # 处理所有垃圾都耗时1个单位时间
        # 计算移动距离
        if i > 0:
            cur_dis += travel[i - 1]
        # 更新不同的垃圾车所移动的距离
        for c in garbage[i]:
            distance[c] = cur_dis
    return res + sum(distance.values())